# MatFormer Lab

`Gemma 3n` là một mô hình đa phương thức (`multimodal`), đa ngôn ngữ (`multilingual`) thuộc họ mô hình `Gemma`. Bạn có thể đọc về `Gemma 3n` trong [tài liệu (docs)](https://www.google.com/search?q=https://ai.google.dev/gemma/docs/gemma2) và [bài đăng blog ra mắt](https://www.google.com/search?q=https://ai.google.dev/blog/gemma-2) của nó. Đây là một mô hình độc đáo có khả năng co giãn tự nhiên (`natively elastic`), nghĩa là bạn sẽ có các mô hình được lồng vào nhau (`nested models`)\! `Gemma 3n` được huấn luyện như một mô hình `E4B` (effectively loads 4B parameters) với 35 lớp và 16,384 FFN hidden dimension. Nó có một mô hình `E2B` (30 lớp và 8,192 FFN hidden dimension) được lồng bên trong và được huấn luyện đồng thời dưới dạng một `MatFormer`.

Kiến trúc [MatFormer](https://arxiv.org/abs/2310.07707) (🪆`Matryoshka Transformer`) là một kiến trúc `transformer` lồng nhau mới lạ được xây dựng cho việc suy luận co giãn (`elastic inference`). Hãy tưởng tượng nó giống như những con búp bê `Matryoshka`: một mô hình lớn hơn chứa các phiên bản nhỏ hơn, đầy đủ chức năng của chính nó. Cách tiếp cận này mở rộng khái niệm của `Matryoshka Representation Learning` từ chỉ các `embeddings` sang tất cả các thành phần của `transformer`.

Việc tiết kiệm bộ nhớ bằng cách lồng `E2B` bên trong `E4B` là rất hữu ích trong thực tế, nhưng điều làm cho `MatFormer` trở nên mạnh mẽ là khả năng của nó có thể trải dài mượt mà trên toàn bộ đường cong tối ưu Pareto về độ chính xác-so với-kích thước mô hình (`Pareto-optimal accuracy-vs-model size curve`) giữa `E2B` và `E4B` mà không cần huấn luyện bổ sung. Sử dụng một kỹ thuật đơn giản gọi là `Mix-n-Match`, người ta có thể trích xuất một mô hình có kích thước bất kỳ giữa `E2B` và `E4B` từ mô hình `E4B` chính.

Tại sao bạn lại muốn "cắt lát" (`slice`) một mô hình? Dựa trên các yêu cầu triển khai (`deployment requirements`) cụ thể của bạn, `E2B` và `E4B` có thể không phải là lựa chọn phù hợp. Ví dụ, bạn có thể muốn có một mô hình `E3B`, với chất lượng cao hơn `E2B` trong khi yêu cầu ít tài nguyên tính toán (`compute`) hơn `E4B`.

Trong `notebook` này, bạn sẽ được thử nghiệm với `MatFormers` và `Mix-n-Match`. Bạn sẽ chỉ định cấu hình mong muốn cho mô hình con (`submodel`) dựa trên chiều `FFN` và các lớp `skip layers`, và sau đó bạn sẽ xuất mô hình sang `Hugging Face`, cho phép bạn sử dụng nó với các công cụ yêu thích của mình.

In [1]:
# @title Install dependencies
# @markdown Run this cell to install all the required dependencies. In particular, you'll need Hugging Face `transformers` and `timm` versions that support Gemma 3n. Note that you may need to restart the notebook after executing the following cell.

# Install a transformers version that supports Gemma 3n (>= 4.53)
!pip install "transformers>=4.53" "timm>=1.0.16" -q


In [2]:

# @title Login to Hugging Face
# @markdown This is required so you can push the model to Hugging Face. You also need to make sure you have access to the Gemma 3n model repositories.

from huggingface_hub import notebook_login

notebook_login()

In [3]:

# @title Import and Export Options
# @markdown The MatFormer Lab allows you to load a Gemma 3n 4B checkpoint (either pre-trained or instruct-tuned) and to slice it. Below, please specify:

# @markdown * The original repository ID from the checkpoint in Hugging Face

# @markdown * A local path where the model will be saved

# @markdown * A name of a repository to push the new checkpoint to

original_model_id = "google/gemma-3n-E4B-it" # @param ["google/gemma-3n-E4B-it", "google/gemma-3n-E4B-pt"]
local_output_path = "my_modified_gemma_3n_model" # @param {type:"string"}
push_hf_repo_id = "ngohongthai/test-submodel"  # @param {type:"string"}

## Cấu hình cắt lát (Slicing configuration)

Là một phần của việc phát hành `Gemma 3n`, chúng tôi chia sẻ các cấu hình cắt lát tối ưu dưới dạng một kho dữ liệu (`dataset repository`) trên **[Hugging Face](https://www.google.com/search?q=https://huggingface.co/datasets/google/gemma-3n-slicing-configs)**, mặc dù bạn cũng có thể tự khám phá các cấu hình của riêng mình ở bên dưới.

Mỗi cấu hình sẽ chỉ định:

  * The hidden dimensions of the FFN
  * Which layers, if any, to skip
  * Độ chính xác `MMLU` tương ứng với các `checkpoint` đã được huấn luyện trước
  
Các cấu hình cấp độ lớp (`layer-level`) và cấp độ khối (`block-level`) là kết quả của việc thay đổi chiều ẩn của `FFN` ở cấp độ lớp (chi tiết - `fine-grained`) hoặc ở cấp độ khối (4 lớp cục bộ + 1 lớp toàn cục).

Ở **cấp độ lớp (`layer-level`)**, chúng tôi nhận thấy rằng việc cho các lớp toàn cục (`global layers`) (thay vì các lớp cục bộ - `local layers`) có năng lực (`capacity`) cao hơn sẽ giúp cải thiện độ chính xác với cùng một kích thước mô hình.

Ở **cấp độ khối (`block-level`)**, chúng tôi thấy rằng khối bị bỏ qua đối với `E2B` (tức là các lớp 20-24) sẽ được hưởng lợi từ năng lực cao hơn khi không bị bỏ qua, và các khối ở tầng sớm hơn có thể hoạt động tốt với năng lực thấp hơn so với các khối ở tầng sau.

Chúng tôi mời cộng đồng cùng tìm ra những cấu hình tốt hơn nữa nằm trên đường cong tối ưu Pareto (`Pareto-optimal curve`) giữa `E2B` và `E4B`\!

In [4]:
import pandas as pd

df = pd.read_csv("hf://datasets/google/gemma3n-slicing-configs/configs.csv")
df.head(20)

,name,# Layers,# Effective Params (B),MMLU PT accuracy,FFN Hidden Dims,Layers Skipped
0,Main model,35,3.98,62.30%,"[2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2...",NaN
1,Config for official E2B Model,30,1.91,50.90%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]"
2,Config for E1.96B (layer-level),30,1.96,53.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]"
3,Config for E2.54B (layer-level),35,2.54,55.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
4,Config for E2.69B (layer-level),35,2.69,57.70%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
5,Config for E2.98B (layer-level),35,2.98,59.50%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
6,Config for E3.18B (layer-level),35,3.18,61.80%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
7,Config for E3.39B (layer-level),35,3.39,63.00%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
8,Config for E3.59B (layer-level),35,3.59,63.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
9,Config for E3.79B (layer-level),35,3.79,63.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN


Based on your deployment scenarios, you may want to pick a different config. Select below your preferred one.

In [5]:
#@title Config details
import ast

config_name = "Config for E1.96B (layer-level)"# @param ['Config for official E2B Model', 'Config for E1.96B (layer-level)', 'Config for E2.54B (layer-level)', 'Config for E2.69B (layer-level)', 'Config for E2.98B (layer-level)', 'Config for E3.18B (layer-level)', 'Config for E3.39B (layer-level)', 'Config for E3.59B (layer-level)', 'Config for E3.79B (layer-level)', 'Config for E2.49B (block-level)', 'Config for E2.73B (block-level)', 'Config for E2.98B (block-level)', 'Config for E3.24B (block-level)', 'Config for E3.49B (block-level)', 'Config for E3.79B (block-level)']


In [6]:
def safe_string_to_list(value):
    """
    Converts a string representation of a list into a Python list.
    - Converts NaN/missing values to an empty list [].
    - Uses eval() to handle expressions like '2_048 * 8'.
    - Safely handles non-string values by returning them as is.
    """
    # First, check if the value is missing (NaN, None, etc.)
    if isinstance(value, list):
        return value

    # Priority 2: Now that we know it's not a list, check if it's a missing value.
    if pd.isna(value):
        return []

    # Priority 3: If it's a string, try to evaluate it.
    if isinstance(value, str):
        try:
            return eval(value)
        except (SyntaxError, NameError):
            return value  # Return invalid string as is

    # Fallback for any other type (like an integer)
    return value


In [7]:
eval('2_048 * 8')

16384

In [8]:
config_name

'Config for E1.96B (layer-level)'

In [9]:
df['FFN Hidden Dims'][0]

'[2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8]'

In [10]:
safe_string_to_list('[20, 21, 22, 23, 24]')

[20, 21, 22, 23, 24]

In [11]:
df['FFN Hidden Dims List'] = df['FFN Hidden Dims'].apply(safe_string_to_list)
df['Layers Skipped'] = df['Layers Skipped'].apply(safe_string_to_list)

In [12]:
df.head()

,name,# Layers,# Effective Params (B),MMLU PT accuracy,FFN Hidden Dims,Layers Skipped,FFN Hidden Dims List
0,Main model,35,3.98,62.30%,"[2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2...",[],"[16384, 16384, 16384, 16384, 16384, 16384, 163..."
1,Config for official E2B Model,30,1.91,50.90%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]","[8192, 8192, 8192, 8192, 8192, 8192, 8192, 819..."
2,Config for E1.96B (layer-level),30,1.96,53.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]","[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."
3,Config for E2.54B (layer-level),35,2.54,55.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",[],"[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."
4,Config for E2.69B (layer-level),35,2.69,57.70%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",[],"[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."


In [13]:
df_indexed = df.set_index('name')
df_indexed.head()

,# Layers,# Effective Params (B),MMLU PT accuracy,FFN Hidden Dims,Layers Skipped,FFN Hidden Dims List
name,,,,,,
Main model,35,3.98,62.30%,"[2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2...",[],"[16384, 16384, 16384, 16384, 16384, 16384, 163..."
Config for official E2B Model,30,1.91,50.90%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]","[8192, 8192, 8192, 8192, 8192, 8192, 8192, 819..."
Config for E1.96B (layer-level),30,1.96,53.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]","[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."
Config for E2.54B (layer-level),35,2.54,55.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",[],"[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."
Config for E2.69B (layer-level),35,2.69,57.70%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",[],"[8192, 8192, 8192, 8192, 16384, 8192, 8192, 81..."


In [14]:
model_row = df_indexed.loc[config_name]
model_row

# Layers                                                                 30
# Effective Params (B)                                                 1.96
MMLU PT accuracy                                                     53.40%
FFN Hidden Dims           [2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...
Layers Skipped                                         [20, 21, 22, 23, 24]
FFN Hidden Dims List      [8192, 8192, 8192, 8192, 16384, 8192, 8192, 81...
Name: Config for E1.96B (layer-level), dtype: object

In [15]:
layers_to_skip = model_row['Layers Skipped']
ffn_hidden_dims = model_row['FFN Hidden Dims List']
ffn_hidden_dims_str = model_row['FFN Hidden Dims']

print(config_name)
print("\nLayers Skipped:")
print(layers_to_skip)
print("\nFFN Hidden Dims:")
print(ffn_hidden_dims_str)

Config for E1.96B (layer-level)

Layers Skipped:
[20, 21, 22, 23, 24]

FFN Hidden Dims:
[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 8, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4]


In [16]:


# Custom config
#
# layers_to_skip = [] # e.g. [20, 21, 22, 23, 24]
# ffn_hidden_dims = [] # e.g. [2048 * 4, ...]
# ffn_hidden_dims_str = str(ffn_hidden_dims)

## Slicing

### Load the model config and verify slicing configuration

Note: we do not load the model at this stage, just verify that the slicing configuration is possible



In [17]:
original_model_id

'google/gemma-3n-E4B-it'

In [18]:
from transformers import AutoConfig, AutoTokenizer

original_config = AutoConfig.from_pretrained(original_model_id)

config.json:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

In [19]:
original_config

Gemma3nConfig {
  "architectures": [
    "Gemma3nForConditionalGeneration"
  ],
  "audio_config": {
    "conf_attention_chunk_size": 12,
    "conf_attention_context_left": 13,
    "conf_attention_context_right": 0,
    "conf_attention_logit_cap": 50.0,
    "conf_conv_kernel_size": 5,
    "conf_num_attention_heads": 8,
    "conf_num_hidden_layers": 12,
    "conf_reduction_factor": 4,
    "conf_residual_weight": 0.5,
    "gradient_clipping": 10000000000.0,
    "hidden_size": 1536,
    "input_feat_size": 128,
    "model_type": "gemma3n_audio",
    "rms_norm_eps": 1e-06,
    "sscp_conv_channel_size": [
      128,
      32
    ],
    "sscp_conv_group_norm_eps": 0.001,
    "sscp_conv_kernel_size": [
      [
        3,
        3
      ],
      [
        3,
        3
      ]
    ],
    "sscp_conv_stride_size": [
      [
        2,
        2
      ],
      [
        2,
        2
      ]
    ],
    "torch_dtype": "bfloat16",
    "vocab_offset": 262272,
    "vocab_size": 128
  },
  "audio_soft_to

In [20]:
model_config = original_config.text_config
model_config

Gemma3nTextConfig {
  "activation_sparsity_pattern": [
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.95,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "altup_active_idx": 0,
  "altup_coef_clip": 120.0,
  "altup_correct_scale": true,
  "altup_num_inputs": 4,
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2048,
  "hidden_size_per_layer_input": 256,
  "initializer_range": 0.02,
  "intermediate_size": [
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,
    16384,

In [21]:
num_layers = model_config.num_hidden_layers
num_layers

35

In [22]:
layers_to_skip

[20, 21, 22, 23, 24]

In [23]:
final_num_layers = num_layers - len(layers_to_skip)
final_num_layers

30

In [24]:
if len(ffn_hidden_dims) != final_num_layers:
    raise ValueError(
        f"The length of ffn_hidden_dims ({len(ffn_hidden_dims)}) must be equal "
        f"to the final number of layers ({final_num_layers})."
    )

### Update configuration

 Khi bạn quyết định "cắt bỏ" một số lớp (ví dụ, bỏ 5 lớp để tạo mô hình 30 lớp), không chỉ số lượng lớp thay đổi, mà các tham số cấu hình khác liên quan đến các lớp đó cũng phải được cập nhật một cách thông minh. Đoạn code này làm chính xác điều đó.

Cập nhật cấu hình chia sẻ Key-Value (KV Sharing):

* **Mục đích**: Gemma 3n sử dụng một kỹ thuật gọi là KV Sharing để tiết kiệm bộ nhớ, trong đó một số lớp sẽ chia sẻ chung các tham số Key (K) và Value (V) trong cơ chế attention. Đoạn code này đảm bảo cấu hình KV Sharing được cập nhật đúng khi các lớp bị loại bỏ.
* **Ý nghĩa các biến**:
    * `model_config.num_hidden_layers`: Tổng số lớp của mô hình gốc (là 35).
    * `model_config.num_kv_shared_layers`: Số lượng lớp chia sẻ KV trong mô hình gốc.
    * `layers_to_skip`: Một danh sách chứa chỉ số của các lớp bạn muốn loại bỏ (ví dụ: `[20, 21, 22, 23, 24]`).
    * `local_kv_sharing_layer_idx` và `global_kv_sharing_layer_idx`: Đây là chỉ số của các lớp rất đặc biệt, được dùng làm "trung tâm" chia sẻ KV. Đoạn `if` đảm bảo rằng bạn không thể vô tình xóa bỏ các lớp quan trọng này.
    * `count_kv_sharing`: Đếm xem có bao nhiêu lớp chia sẻ KV (các lớp từ 20 trở đi) đã bị bạn loại bỏ.
    * `model_config.num_kv_shared_layers -= count_kv_sharing`: Cập nhật lại tổng số lớp chia sẻ KV trong mô hình mới sau khi đã trừ đi các lớp bị loại bỏ.

In [ ]:
# Tính toán các chỉ số của các lớp đặc biệt
num_kv_comp_layers = model_config.num_hidden_layers - model_config.num_kv_shared_layers
local_kv_sharing_layer_idx = num_kv_comp_layers - 2
global_kv_sharing_layer_idx = num_kv_comp_layers - 1

# Kiểm tra xem các lớp đặc biệt có bị bỏ qua hay không
if (local_kv_sharing_layer_idx in layers_to_skip or global_kv_sharing_layer_idx in layers_to_skip):
  raise ValueError(f'Layers {local_kv_sharing_layer_idx} and {global_kv_sharing_layer_idx} are reserved.')

# Đếm và cập nhật lại số lớp chia sẻ KV
count_kv_sharing = sum(1 for layer in layers_to_skip if layer >= 20)
model_config.num_kv_shared_layers -= count_kv_sharing


In [26]:
original_config.text_config.num_kv_shared_layers

10

 Cập nhật cấu hình độ thưa của kích hoạt (Activation Sparsity):

* **Mục đích**: Gemma 3n áp dụng "độ thưa" (sparsity) cho các hàm kích hoạt ở 10 lớp đầu tiên để tăng hiệu quả tính toán. Khi bạn xóa một trong số các lớp này, mô hình độ thưa cũng phải được điều chỉnh tương ứng.
* **Ý nghĩa các biến**:
    * `count_activation_sparsity`: Đếm xem có bao nhiêu lớp trong 10 lớp đầu tiên đã bị bạn loại bỏ.
    * `final_num_layers`: Tổng số lớp của mô hình *sau khi* đã cắt bỏ.
    * `activation_sparsity_list`: Tạo ra một danh sách mới. Danh sách này xác định mô hình độ thưa (sparsity pattern) cho các lớp còn lại, đảm bảo rằng cấu trúc này vẫn được duy trì một cách chính xác trong mô hình con.

In [27]:
# Đếm số lớp có độ thưa bị loại bỏ
count_activation_sparsity = sum(1 for layer in layers_to_skip if layer <= 9)

# Tạo lại danh sách mô hình độ thưa cho các lớp còn lại
activation_sparsity_list = [0.95] * (10 - count_activation_sparsity) + [0] * (
    final_num_layers - 10 + count_activation_sparsity
)
model_config.activation_sparsity_pattern = activation_sparsity_list

In [28]:
model_config.num_hidden_layers = final_num_layers
model_config.intermediate_size = ffn_hidden_dims

### Save the configuration and the unchanged tokenizer

In [29]:
original_config.save_pretrained(local_output_path)
tokenizer = AutoTokenizer.from_pretrained(original_model_id)
tokenizer.save_pretrained(local_output_path)

print(f"New config saved to {local_output_path}")
print(f"Final number of layers: {model_config.num_hidden_layers}")

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

New config saved to my_modified_gemma_3n_model
Final number of layers: 30


### Load the model checkpoints

Note: we are saving the model to disk, so there's no need to have a large CPU/GPU.

In [30]:
import os

from huggingface_hub import snapshot_download

model_path = snapshot_download(original_model_id, allow_patterns=["*.safetensors"])
safetensor_files = [os.path.join(model_path, f) for f in os.listdir(model_path) if f.endswith('.safetensors')]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

### Slice the model!

In [31]:
from safetensors import safe_open
from tqdm.auto import tqdm
import re
import torch
import gc

from safetensors.torch import save_file

kept_layers_indices = [i for i in range(num_layers) if i not in layers_to_skip]
layer_rename_map = {old_idx: new_idx for new_idx, old_idx in enumerate(kept_layers_indices)}

# This will store the mapping of tensor names to the file they are saved in
weight_map = {}

# This will store tensors for the current shard we are building
new_shard_state_dict = {}
shard_counter = 1
total_size = 0

pbar = tqdm(total=len(safetensor_files), desc="Processing shards")

for shard_path in safetensor_files:
    # Open a shard for streaming
    with safe_open(shard_path, framework="pt", device="cpu") as f:
        # Iterate over each tensor in the shard
        for tensor_name in f.keys():
            new_tensor_name = tensor_name
            tensor = f.get_tensor(tensor_name)

            # Case 1: Handle layer-specific parameters
            match = re.search(r'\.layers\.(\d+)\.', tensor_name)
            if match:
                old_layer_idx = int(match.group(1))

                # If this layer is meant to be skipped, we just continue to the next tensor
                if old_layer_idx in layers_to_skip:
                    continue

                # Get the new sequential layer index
                new_layer_idx = layer_rename_map[old_layer_idx]
                new_tensor_name = tensor_name.replace(
                    f'.layers.{old_layer_idx}.',
                    f'.layers.{new_layer_idx}.'
                )

                # Get the target FFN dimension for this new layer
                target_ffn_dim = ffn_hidden_dims[new_layer_idx]

                # Check if this parameter is part of the FFN and needs slicing
                if 'mlp.gate_proj.weight' in new_tensor_name or 'mlp.up_proj.weight' in new_tensor_name:
                    # These layers project from model_dim -> ffn_hidden_dim.
                    # We slice the output dimension (dim 0).
                    tensor = tensor[:target_ffn_dim, :].contiguous()
                elif 'mlp.down_proj.weight' in new_tensor_name:
                    # This layer projects from ffn_hidden_dim -> model_dim.
                    # We slice the input dimension (dim 1).
                    tensor = tensor[:, :target_ffn_dim].contiguous()

            # Case 2: Handle special non-layer parameters that need slicing
            elif 'per_layer_model_projection' in tensor_name:
                # Reshape, slice based on kept layers, and reshape back
                reshaped_params = tensor.reshape((num_layers, tensor.shape[0] // num_layers, tensor.shape[1]))
                tensor = reshaped_params[kept_layers_indices, :, :]
                tensor = tensor.reshape(-1, tensor.shape[-1]).contiguous()

            elif 'embed_tokens_per_layer' in tensor_name:
                # Reshape, slice based on kept layers, and reshape back
                reshaped_params = tensor.reshape((tensor.shape[0], num_layers, tensor.shape[1] // num_layers))
                tensor = reshaped_params[:, kept_layers_indices, :]
                tensor = tensor.reshape(tensor.shape[0], -1).contiguous()

            # Add the (potentially modified) tensor to the new shard
            new_shard_state_dict[new_tensor_name] = tensor

            # Check if the current shard is getting too big
            current_shard_size = sum(t.numel() * t.element_size() for t in new_shard_state_dict.values())
            if current_shard_size > 4000000000: # Create new shard if current is over 4GB
                shard_filename = f"model-{(shard_counter):05d}-of-XXXXX.safetensors"
                print(f"Saving shard {shard_filename} (size: {current_shard_size / 1e9:.2f} GB)")
                save_file(new_shard_state_dict, os.path.join(local_output_path, shard_filename), metadata={'format': 'pt'})

                # Record which tensors are in this shard
                for k in new_shard_state_dict.keys():
                    weight_map[k] = os.path.basename(shard_filename)

                # Reset for the next shard
                shard_counter += 1
                new_shard_state_dict = {}
                gc.collect() # Free up memory
    pbar.update(1)

pbar.close()

Processing shards:   0%|          | 0/4 [00:00<?, ?it/s]

Saving shard model-00001-of-XXXXX.safetensors (size: 4.86 GB)
Saving shard model-00002-of-XXXXX.safetensors (size: 4.63 GB)


In [32]:
# Save any remaining tensors in the last shard
if new_shard_state_dict:
    shard_filename = f"model-{(shard_counter):05d}-of-XXXXX.safetensors"
    print(f"Saving final shard {shard_filename}")
    save_file(new_shard_state_dict, os.path.join(local_output_path, shard_filename), metadata={'format': 'pt'})
    for k in new_shard_state_dict.keys():
        weight_map[k] = os.path.basename(shard_filename)

Saving final shard model-00003-of-XXXXX.safetensors


In [33]:
del new_shard_state_dict
gc.collect()

31

In [34]:
import json
print("\n--- 3. Finalizing Model Save ---")

# The total number of shards we created
num_shards = shard_counter

# Update the "XXXXX" in the filenames to the correct total number of shards
for i in range(1, num_shards + 1):
    old_filename = f"model-{(i):05d}-of-XXXXX.safetensors"
    new_filename = f"model-{(i):05d}-of-{(num_shards):05d}.safetensors"

    # Rename the file
    os.rename(os.path.join(local_output_path, old_filename), os.path.join(local_output_path, new_filename))

    # Update the weight_map to point to the new filename
    for k, v in weight_map.items():
        if v == old_filename:
            weight_map[k] = new_filename

# Create and save the index.json file
index_json = {
    "metadata": {
        "total_size": sum(os.path.getsize(os.path.join(local_output_path, f)) for f in os.listdir(local_output_path) if f.endswith('.safetensors'))
    },
    "weight_map": weight_map
}

with open(os.path.join(local_output_path, "model.safetensors.index.json"), "w") as f:
    json.dump(index_json, f, indent=2)

print(f"\n✅ Model slicing complete. New model saved in: {local_output_path}")


--- 3. Finalizing Model Save ---

✅ Model slicing complete. New model saved in: my_modified_gemma_3n_model


In [35]:
from huggingface_hub import  ModelCard, ModelCardData

card = ModelCard.load(original_model_id)
card.data.base_model = original_model_id
del card.data.extra_gated_heading
del card.data.extra_gated_prompt
card.data.tags.append("matformer")

new_description = f"""
> [!Note]
> This is a submodel derived from `{original_model_id}`. It has been modified by slicing specific layers and resizing FFN dimensions. It is not the original model.
> To learn more about MatFormers, please review the [launch blog](https://developers.googleblog.com/en/introducing-gemma-3n-developer-guide) and generate your own submodels
with the [MatFormer Lab](https://goo.gle/gemma3n-matformer-lab).
>

Skipped layers: {layers_to_skip}

FFN hidden dimensions: {ffn_hidden_dims_str}
"""

card.text = new_description + "\n" + card.text
print("Prepended custom description to the model card content.")

new_readme_path = os.path.join(local_output_path, "README.md")
card.save(new_readme_path)
print(f"New README.md saved to '{new_readme_path}'")

README.md:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Prepended custom description to the model card content.
New README.md saved to 'my_modified_gemma_3n_model/README.md'


## Push the model to Hugging Face

In [36]:
from huggingface_hub import HfApi

print(f"Creating private repository: {push_hf_repo_id}")

# Instantiate the HfApi client
api = HfApi()

# Create a new private repository on the Hub.
repo_url = api.create_repo(
    repo_id=push_hf_repo_id,
    private=True,
    exist_ok=True
)

Creating private repository: ngohongthai/test-submodel


In [37]:
print(f"Uploading files from '{local_output_path}' to '{push_hf_repo_id}'...")
api.upload_folder(
    folder_path=local_output_path,
    repo_id=push_hf_repo_id,
    repo_type="model",
    commit_message="Upload sliced model checkpoint"
)

Uploading files from 'my_modified_gemma_3n_model' to 'ngohongthai/test-submodel'...


model-00001-of-00003.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ngohongthai/test-submodel/commit/4085e6ca1324781669801266f9ffe84f44171fd0', commit_message='Upload sliced model checkpoint', commit_description='', oid='4085e6ca1324781669801266f9ffe84f44171fd0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ngohongthai/test-submodel', endpoint='https://huggingface.co', repo_type='model', repo_id='ngohongthai/test-submodel'), pr_revision=None, pr_num=None)

## Verify new model can be loaded

In [39]:
#@title Verify new model can be loaded


from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(push_hf_repo_id, torch_dtype=torch.bfloat16, device_map="auto")

print(f"Total Parameters: {model.num_parameters():,}") # 5,976,833,408
print(f"Total Text Parameters: {model.language_model.num_parameters():,}") # 4,456,156,768
print(f"Effective Parameters (excluding vision, audio, and Per-Layer-Embeddings): {model.language_model.num_parameters(exclude_embeddings=True):,}") # 1,905,495,648

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


Total Parameters: 6,027,165,120
Total Text Parameters: 4,506,488,416
Effective Parameters (excluding vision, audio, and Per-Layer-Embeddings): 1,955,827,296
